# AnnData objects between python and R

In [6]:
import scanpy as sc
import numpy as np
import os
import pickle
from re import sub
import anndata2ri

In [2]:
def load_object(file):
    with open(file, 'rb') as inF:
        obj = pickle.load(inF)
        return(obj)
    
def absolute_paths(directory):
    for dirpath,_,filenames in os.walk(directory):
        for f in filenames:
            yield os.path.abspath(os.path.join(dirpath, f))

### Write AnnData to loom in python

In [23]:
adata_dir = "/Users/larisamorales/OneDrive - McGill University/scvelo/adata"
for obj in absolute_paths(adata_dir):
    adata = load_object(obj)
    
    for k in adata.layers.keys():
        np.nan_to_num(adata.layers[k],copy=False)
    
    adata.write_loom(sub("pkl","loom",obj),write_obsm_varm=True)

### Load loom in R

In [34]:
%%R 

# Installations 

#devtools::install_github(repo = "hhoeflin/hdf5r")
#devtools::install_github(repo = "mojaveazure/loomR", ref = "develop")

# Load packages 

library(loomR)

# Read looms

loom_dir <- "/Users/larisamorales/OneDrive - McGill University/scvelo/adata"
looms <- list.files(loom_dir,pattern = 'loom',full.names = T)
lfile <- connect(filename = looms[1], mode = "r+",skip.validate = T)
lfile

Class: loom
Filename: /Users/larisamorales/OneDrive - McGill University/scvelo/adata/hFB18-adata.loom
Access type: H5F_ACC_RDWR
Attributes: last_modified, version
Listing:
       name    obj_type dataset.dims dataset.type_class
      attrs   H5I_GROUP         <NA>               <NA>
  col_attrs   H5I_GROUP         <NA>               <NA>
 col_graphs   H5I_GROUP         <NA>               <NA>
     layers   H5I_GROUP         <NA>               <NA>
     matrix H5I_DATASET  1720 x 2000          H5T_FLOAT
  row_attrs   H5I_GROUP         <NA>               <NA>
 row_graphs   H5I_GROUP         <NA>               <NA>


In [36]:
%%R 
lfile$close_all()